In [1]:
# Install required libraries
!pip install transformers sentence-transformers faiss-cpu wikipedia --quiet


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 81.1 MB/s eta 0:00:00


In [2]:
# Import necessary libraries

from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import wikipedia


In [3]:
# Load a free HuggingFace text generation model
# This model runs on GPU if available

generator = pipeline(
    "text-generation",
    model="google/flan-t5-base",
    device=0  # 0 = GPU, -1 = CPU
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausa

In [4]:
# Short-term memory (stores recent conversations)

short_term_memory = []

def add_to_memory(user, response):
    short_term_memory.append({"user": user, "response": response})

    # Keep only last 5 conversations
    if len(short_term_memory) > 5:
        short_term_memory.pop(0)


In [5]:
# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Create FAISS index
dimension = 384  # embedding size of model
index = faiss.IndexFlatL2(dimension)

long_term_texts = []

def add_long_term_memory(text):
    embedding = embedding_model.encode([text])
    index.add(np.array(embedding))
    long_term_texts.append(text)

def search_long_term_memory(query):
    query_embedding = embedding_model.encode([query])
    D, I = index.search(np.array(query_embedding), k=1)

    if len(long_term_texts) > 0:
        return long_term_texts[I[0][0]]
    return ""


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def calculator_tool(expression):
    try:
        return str(eval(expression))
    except:
        return "Invalid calculation"


In [7]:
def wiki_tool(query):
    try:
        return wikipedia.summary(query, sentences=2)
    except:
        return "No information found."


In [8]:
def agent_reasoning(user_input):

    print("🧠 Reasoning Step:")

    # If math detected → use calculator
    if any(char.isdigit() for char in user_input):
        print("→ Detected mathematical expression. Using calculator tool.")
        return calculator_tool(user_input)

    # If user asks about 'who', 'what', etc → use wiki
    elif any(word in user_input.lower() for word in ["who", "what", "where", "when"]):
        print("→ Detected knowledge query. Using Wikipedia tool.")
        return wiki_tool(user_input)

    else:
        print("→ Using LLM for general response.")
        context = search_long_term_memory(user_input)

        prompt = f"""
        Context: {context}
        User: {user_input}
        Answer:
        """

        result = generator(prompt, max_length=200)
        return result[0]['generated_text']


In [9]:
def intelligent_agent(user_input):

    # Get response
    response = agent_reasoning(user_input)

    # Save in short-term memory
    add_to_memory(user_input, response)

    # Save important things to long-term memory
    add_long_term_memory(user_input + " " + response)

    return response


In [10]:
print(intelligent_agent("What is Artificial Intelligence?"))


🧠 Reasoning Step:
→ Detected knowledge query. Using Wikipedia tool.
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.


In [11]:
print(intelligent_agent("2 + 56 * 3"))


🧠 Reasoning Step:
→ Detected mathematical expression. Using calculator tool.
170


In [12]:
print(intelligent_agent("Who is Elon Musk?"))


🧠 Reasoning Step:
→ Detected knowledge query. Using Wikipedia tool.
Errol Graham Musk (born 25 May 1946) is a South African businessman, politician, and the patriarch of the Musk family. He was an Independent member of Pretoria City Council from 1972 to 1980 and then as a member of Progressive Federal Party from 1980 to 1983.
